In [2]:
import re
import pandas as pd
import funcs 
import importlib
import unicodecsv as csv
import pickle
import random
from timeit import default_timer as timer
from datetime import datetime, timedelta
import ng_models
import matplotlib.pyplot as plt
from collections import Counter

%matplotlib notebook
importlib.reload(funcs)
importlib.reload(ng_models)

<module 'ng_models' from 'D:\\_projekty\\praca\\ng_models.py'>

In [4]:
len(sp_joined)


158885

In [3]:
sp_joined = pd.read_csv('00_dane/sp_joined.csv',  sep='|', encoding='UTF-8')


Wgranie pliku przygotowanego w SQL, zawierającego najnowsze przypisanie do partii.

In [5]:
parties = pd.read_csv('00_dane/party.csv',  sep='|', encoding='UTF-8')

In [62]:
parties.head(5)

,full_name,term,party,party_final
0,Adam Abramowicz,8,Klub Parlamentarny Prawo i Sprawiedliwość,Prawo i Sprawiedliwość
1,Adam Andruszkiewicz,8,Poseł niezrzeszony,posłowie niezrzeszeni
2,Adam Babul,0,Klub Poselski Polskiego Stronnictwa Ludowego,Polskie Stronnictwo Ludowe
3,Adam Biela,3,Klub Parlamentarny Akcji Wyborczej Solidarność\n,Akcja Wyborcza Solidarność
4,Adam Bielan,4,Klub Parlamentarny Prawo i Sprawiedliwość\n,Prawo i Sprawiedliwość


In [6]:
sp_joined=sp_joined.merge(parties,how='left',left_on=['author_final'],right_on=['full_name'])

In [8]:
sp_joined.head(20)

,Unnamed: 0,date_,author_final,speech_title,speech_raw,speech_rawer,liczba_sl_raw,liczba_sl_rawer,full_name,term,party,party_final
0,0,01-02-1992,Aleksander Bentkowski,Sprawozdanie Komisji Sprawiedliwości oraz Komi...,Panie Marszałku! Wysoki Sejmie! Chciałbym kil...,Chciałbym kilka zdań powiedzieć na temat skut...,851,845,Aleksander Bentkowski,3.0,Klub Parlamentarny Polskiego Stronnictwa Ludow...,Polskie Stronnictwo Ludowe
1,1,01-02-1992,Aleksander Kwaśniewski,Sprawozdanie Komisji Sprawiedliwości oraz Komi...,Panie Marszałku! Wysoka Izbo! Przysłuchując s...,"Przysłuchując się tej bardzo emocjonalnej, dł...",919,915,Aleksander Kwaśniewski,2.0,Sojusz Lewicy Demokratycznej Klub Parlamentarny,Sojusz Lewicy Demokratycznej
2,2,01-02-1992,Alojzy Pietrzyk,Sprawozdanie Komisji Sprawiedliwości oraz Komi...,Panie Marszałku! Wysoka Izbo! Różne przebyliś...,"Różne przebyliśmy drogi, żeby tu dzisiaj powi...",329,325,Alojzy Pietrzyk,1.0,Klub Parlamentarny NSZZ Solidarność,Klub Parlamentarny NSZZ Solidarność
3,3,01-02-1992,Alojzy Szablewski,Oświadczenia.,Panie Marszałku! Wysoka Izbo! Wszystkim nam b...,"Wszystkim nam bardzo zależy na tym, aby urato...",611,607,Alojzy Szablewski,1.0,Klub Parlamentarny Zjednoczenia Chrześcijańsko...,Klub Parlamentarny Zjednoczenia Chrześcijańsko...
4,4,01-02-1992,Andrzej Baraniecki,Sprawozdanie Komisji Sprawiedliwości oraz Komi...,Wysoka Izbo! Z brzmienia pkt. 1 projektu uchw...,Z brzmienia pkt. 1 projektu uchwały odczytuję...,130,128,Andrzej Baraniecki,1.0,Sojusz Lewicy Demokratycznej - Klub Parlamentarny,Sojusz Lewicy Demokratycznej
5,5,01-02-1992,Andrzej Potocki,Sprawozdanie Komisji Sprawiedliwości oraz Komi...,Panie Marszałku! Wysoka Izbo! W związku z tym...,W związku z tym chcę zapytać panią Ponieważ S...,39,33,Andrzej Potocki,3.0,Klub Parlamentarny Unii Wolności \n,Klub Parlamentarny Unii Wolności \n
6,6,01-02-1992,Andrzej Tadeusz Mazurkiewicz,Sprawozdanie Komisji Sprawiedliwości oraz Komi...,"Andrzej Tadeusz Mazurkiewicz, Konfederacja Po...","Andrzej Tadeusz Mazurkiewicz, Konfederacja Po...",211,207,NaN,NaN,NaN,NaN
7,7,01-02-1992,Andrzej Urbański,Sprawozdanie Komisji Sprawiedliwości oraz Komi...,Panie Marszałku! Panie! Panowie! Parlamentarn...,Parlamentarny Klub Porozumienia Centrum pop...,736,732,Andrzej Urbański,1.0,Klub Polski Program Liberalny,Klub Polski Program Liberalny
8,8,01-02-1992,Andrzej Wojtyła,Sprawozdanie Komisji Sprawiedliwości oraz Komi...,Panie Marszałku! Wysoka Izbo! W nocy z 12 na ...,W nocy z 12 na 13 grudnia 1981 r. ówczesne ko...,462,450,Andrzej Wojtyła,4.0,Koło Poselskie Konserwatywno-Ludowe\n,Koło Poselskie Konserwatywno-Ludowe\n
9,9,01-02-1992,Andrzej Zarębski,Sprawozdanie Komisji Sprawiedliwości oraz Komi...,Panie Marszałku! Panie i Panowie Posłowie! Kl...,Klub Liberalno-Demokratyczny popiera projekt ...,793,787,Andrzej Zarębski,1.0,Klub Polski Program Liberalny,Klub Polski Program Liberalny


Zapisanie pliku z dodaną partią.

In [69]:
sp_joined.to_csv('00_dane/sp_joined2.csv', encoding='UTF-8', index=False, sep="|")

#### Które partie mają nawięcej treści?

In [5]:
ag=sp_joined.groupby('party_final').agg({'liczba_sl_raw': sum, 'date_': 'count'}).reset_index()


In [6]:
ag.sort_values('date_', ascending=False).head(7)

,party_final,liczba_sl_raw,date_
58,Prawo i Sprawiedliwość,12409825,31630
60,Sojusz Lewicy Demokratycznej,13143036,27129
56,Platforma Obywatelska,9438332,24977
57,Polskie Stronnictwo Ludowe,8256113,17851
62,posłowie niezrzeszeni,3735227,7358
0,Akcja Wyborcza Solidarność,3304864,5642
59,Samoobrona,2027951,4407


## Budowa modeli dedykowanych poszczególnym partiom

Czy wypowiedzi są podobne?

In [3]:
PiS = ' '.join(sp_joined['speech_raw'][sp_joined.party_final=='Prawo i Sprawiedliwość'])
PO = ' '.join(sp_joined['speech_raw'][sp_joined.party_final=='Platforma Obywatelska'])
SLD= ' '.join(sp_joined['speech_raw'][sp_joined.party_final=='Sojusz Lewicy Demokratycznej'])
PSL = ' '.join(sp_joined['speech_raw'][sp_joined.party_final=='Polskie Stronnictwo Ludowe'])
PNZ = ' '.join(sp_joined['speech_raw'][sp_joined.party_final=='posłowie niezrzeszeni'])
AWS = ' '.join(sp_joined['speech_raw'][sp_joined.party_final=='Akcja Wyborcza Solidarność'])
SO = ' '.join(sp_joined['speech_raw'][sp_joined.party_final=='Samoobrona'])


### Czy modele wygenerują się wystarczająco szybko?

In [4]:
%%time
model_PiS = ng_models.ng_models(1, PiS)
model_PO = ng_models.ng_models(1, PO)
model_SLD = ng_models.ng_models(1, SLD)
model_PSL = ng_models.ng_models(1, PSL)
model_PNZ = ng_models.ng_models(1, PNZ)
model_AWS = ng_models.ng_models(1, AWS)
model_SO = ng_models.ng_models(1, SO)

Wall time: 3min 1s


In [22]:
%%time
model_PiS = ng_models.ng_models(2, PiS)
model_PO = ng_models.ng_models(2, PO)
model_SLD = ng_models.ng_models(2, SLD)
model_PSL = ng_models.ng_models(2, PSL)
model_PNZ = ng_models.ng_models(2, PNZ)
model_AWS = ng_models.ng_models(2, AWS)
model_SO = ng_models.ng_models(2, SO)

#dla 1-gram 2 min 44 sek 9na nowym kompie 1 min 15 s
# dla 2 gram Wall time: 2min 22s

Wall time: 2min 22s


In [17]:
summary_table = pd.DataFrame()
summary_table['Zdanie']=['Zapewniam państwa, że jestem uczciwym politykiem', 
                        'Lew czarownica i stara szafa',
                        'Zupełnie przypadkowy układ pięciu słów',
                        'Panie ministrze proszę o wysłuchanie',
                        'Polskie rolnictwo rozwija się szybko i ma się dobrze',
                        'Demokracja jest najlepszym systemem politycznym',
                        'Jestem najlepszym kandydatem na prezydenta',
                        'Dla porównania zdanie długie o polityce o ustawie dotyczące zagadnień istotnych, które każdy poseł wypowiedzieć może',
                        'wysoka izbo']

In [18]:
summary_table['PiS'] = summary_table['Zdanie'].apply(model_PiS.perplexity).round()
summary_table['PO'] = summary_table['Zdanie'].apply(model_PO.perplexity).round()
summary_table['SLD'] = summary_table['Zdanie'].apply(model_SLD.perplexity).round()
summary_table['PSL'] = summary_table['Zdanie'].apply(model_PSL.perplexity).round()
summary_table['PNZ'] = summary_table['Zdanie'].apply(model_PNZ.perplexity).round()
summary_table['AWS'] = summary_table['Zdanie'].apply(model_AWS.perplexity).round()
summary_table['SO'] = summary_table['Zdanie'].apply(model_SO.perplexity).round()

In [19]:
summary_table.set_index('Zdanie').sort_values('PiS')

,PiS,PO,SLD,PSL,PNZ,AWS,SO
Zdanie,,,,,,,
wysoka izbo,55.0,66.0,79.0,85.0,108.0,203.0,161.0
Panie ministrze proszę o wysłuchanie,9793.0,8982.0,13313.0,12065.0,22204.0,30020.0,9233.0
Polskie rolnictwo rozwija się szybko i ma się dobrze,162484.0,183243.0,139085.0,114539.0,172099.0,178610.0,119240.0
"Dla porównania zdanie długie o polityce o ustawie dotyczące zagadnień istotnych, które każdy poseł wypowiedzieć może",268838.0,246964.0,264019.0,220361.0,162338.0,151278.0,134367.0
Jestem najlepszym kandydatem na prezydenta,491029.0,576307.0,597829.0,437984.0,372432.0,383842.0,305622.0
"Zapewniam państwa, że jestem uczciwym politykiem",522416.0,579467.0,675502.0,515996.0,368037.0,345055.0,463992.0
Demokracja jest najlepszym systemem politycznym,1341180.0,1514168.0,1464463.0,1252672.0,824512.0,827542.0,859406.0
Zupełnie przypadkowy układ pięciu słów,1569920.0,1396201.0,1409566.0,1137613.0,886868.0,756022.0,730734.0
Lew czarownica i stara szafa,3106221.0,2541750.0,2025807.0,2645245.0,1355215.0,1402827.0,987178.0


In [84]:
model_PO.model_dict['test z']

['odwagi', 'biologii']

In [23]:
model_PO.generate(steps=10, max_sent=10)

'Proponowane przez rząd projekt ustawy był przygotowany przez poprzedni rząd. Dobrze ponad 50% mandatów w sejmie był forsowany przede wszystkim. Chciałbym przeprosić pana ministra o wyjaśnienie waszej strategii politycznej bo. Dzisiaj ustosunkować do apeli osób głuchoniemych dotyczących sztucznego systemu językowo-migowego. Natomiast wybory samorządowe dlatego też możliwość przedłużenia funkcjonowania podatku importowego. Z playbacku. Nie utrudnią nam szukania tego co wiadomo z którym zwracały. Nie zgodnie z art. Panie piotrowicz do sytuacji w praktyce w tym sejmie na. Nie zabiegać.'

In [24]:
model_SO.generate(steps=10, max_sent=10)

'Prezydent złamał konstytucję a składał przysięgę że będzie też źle. Po najniższych cenach natomiast ten kto dysponuje mediami obok zarabiania. Wywiadów środowiskowych. Nieszczęściem w postaci podatków zapłacą wasze dzieci i walczyć o. Wysoki koszt kredytu inwestycyjnego zaciągniętego przez kasy chorych musi mu. W posiedzeniu komisji gospodarki o rządowym projekcie ustawy o zmianie. Zdziwienie i twierdzi że możliwa tu będzie. I nagłaśniane przez media od lat popieramy wszystkie racjonalne inicjatywy. Można wyrobić zysk z pracy. Wysoka cena będą odpowiedzialne za dopuszczenie do obciążenia podatkowymi kosztami.'

## Najbardziej prawdopodobne zdanie dla danej partii

Wykorzystuję bardziej oczyszczony tekst aby nie wpadać ciagne na zbitki typu "Wysoka Izbo!".

In [15]:
PiS = ' '.join(sp_joined['speech_rawer'][sp_joined.party_final=='Prawo i Sprawiedliwość'])
PO = ' '.join(sp_joined['speech_rawer'][sp_joined.party_final=='Platforma Obywatelska'])
SLD= ' '.join(sp_joined['speech_rawer'][sp_joined.party_final=='Sojusz Lewicy Demokratycznej'])
PSL = ' '.join(sp_joined['speech_rawer'][sp_joined.party_final=='Polskie Stronnictwo Ludowe'])
PNZ = ' '.join(sp_joined['speech_rawer'][sp_joined.party_final=='posłowie niezrzeszeni'])
AWS = ' '.join(sp_joined['speech_rawer'][sp_joined.party_final=='Akcja Wyborcza Solidarność'])
SO = ' '.join(sp_joined['speech_rawer'][sp_joined.party_final=='Samoobrona'])


In [16]:
PiS[:1000]

' Podobnie jak wielu moich przedmówców, nie chciałem zabierać głosu w tej rzeczywiście mocno za długiej dyskusji. Zdecydowałem się tylko dlatego, że jak mówił Jacek Kuroń próbowano tutaj zaprzeczać faktom, próbowano przedstawić stan wojenny jako zabieg ratowania kraju przed obcą inwazją, przed interwencją sowiecką. Otóż można by sobie wyobrazić taką sytuację. Można by sobie wyobrazić, że Jaruzelski wraz z towarzyszami w tajemnicy przed Moskwą przygotowuje stan wojenny, żeby ochronić Polskę w ostatniej chwili, kiedy już dywizje sowieckie będą skoncentrowane i gotowe do wymarszu, kiedy będzie można rzeczywiście mówić, że to jest mniejsze zło. Można to sobie wyobrazić i wypowiedzi, w których mówiono o tym, że w gruncie rzeczy w żadnym wypadku państwo nie może użyć wobec własnego społeczeństwa przymusu są, jak sądzę, niesłuszne. Są takie wypadki i mam nadzieję, że one się już nigdy w polskiej historii nie powtórzą, ale można je sobie co najmniej wyobrazić. Natomiast w wypadku 13 grudnia, o

In [17]:
%%time
model_PiS = ng_models.ng_models(1, PiS)
model_PO = ng_models.ng_models(1, PO)
model_SLD = ng_models.ng_models(1, SLD)
model_PSL = ng_models.ng_models(1, PSL)
model_PNZ = ng_models.ng_models(1, PNZ)
model_AWS = ng_models.ng_models(1, AWS)
model_SO = ng_models.ng_models(1, SO)


Wall time: 3min 19s


In [46]:
%%time
model_PiS = ng_models.ng_models(2, PiS)
model_PO = ng_models.ng_models(2, PO)
model_SLD = ng_models.ng_models(2, SLD)
model_PSL = ng_models.ng_models(2, PSL)
model_PNZ = ng_models.ng_models(2, PNZ)
model_AWS = ng_models.ng_models(2, AWS)
model_SO = ng_models.ng_models(2, SO)

Wall time: 4min 41s


In [47]:
max_sent = pd.DataFrame()
max_sent['start']=['Ja nie', 'Dziś jest', 'Czy już','Nie można', '']
max_sent['PiS'] = max_sent['start'].apply(model_PiS.mprob_sent)
max_sent['PO'] = max_sent['start'].apply(model_PO.mprob_sent)
max_sent['SLD'] = max_sent['start'].apply(model_SLD.mprob_sent)
max_sent['PSL'] = max_sent['start'].apply(model_PSL.mprob_sent)
max_sent['PNZ'] = max_sent['start'].apply(model_PNZ.mprob_sent)
max_sent['AWS'] = max_sent['start'].apply(model_AWS.mprob_sent)
max_sent['SO'] = max_sent['start'].apply(model_SO.mprob_sent)

max_sent.set_index('start')

1  <s> <s> ja nie
2  ja nie
3  wiem
1  <s> <s> dziś jest
2  dziś jest
3  to
1  <s> <s> czy już
2  czy już
3  nie
1  <s> <s> nie można
2  nie można
3  było
1  <s> <s> 
2  <s> <s>
3  w
1  <s> <s> ja nie
2  ja nie
3  wiem
1  <s> <s> dziś jest
2  dziś jest
3  to
1  <s> <s> czy już
2  czy już
3  w
1  <s> <s> nie można
2  nie można
3  było
1  <s> <s> 
2  <s> <s>
3  w
1  <s> <s> ja nie
2  ja nie
3  jestem
1  <s> <s> dziś jest
2  dziś jest
3  to
1  <s> <s> czy już
2  czy już
3  w
1  <s> <s> nie można
2  nie można
3  było
1  <s> <s> 
2  <s> <s>
3  w
1  <s> <s> ja nie
2  ja nie
3  będę
1  <s> <s> dziś jest
2  dziś jest
3  to
1  <s> <s> czy już
2  czy już
3  jest
1  <s> <s> nie można
2  nie można
3  było
1  <s> <s> 
2  <s> <s>
3  w
1  <s> <s> ja nie
2  ja nie
3  jestem
1  <s> <s> dziś jest
2  dziś jest
3  to
1  <s> <s> czy już
2  czy już
3  po
1  <s> <s> nie można
2  nie można
3  było
1  <s> <s> 
2  <s> <s>
3  w
1  <s> <s> ja nie
2  ja nie
3  znam
1  <s> <s> dziś jest
2  dziś jest
3  bardzo
1  <s

,PiS,PO,SLD,PSL,PNZ,AWS,SO
start,,,,,,,
Ja nie,Ja nie wiem czy pan minister powiedział że w tym zakresie.,Ja nie wiem czy pan minister powiedział że w tym zakresie.,Ja nie jestem w stanie odpowiedzieć na pytanie czy nie uważa pan że.,Ja nie będę się odnosił do tego że w tym zakresie.,Ja nie jestem w stanie tego zrobić.,Ja nie znam.,Ja nie chcę się odnieść do wypowiedzi pana posła sprawozdawcy.
Dziś jest,Dziś jest to że w tym zakresie.,Dziś jest to że w tym zakresie.,Dziś jest to że w tym zakresie.,Dziś jest to że w tym zakresie.,Dziś jest to że w tej sprawie.,Dziś jest bardzo ważne.,Dziś jest już w tej sprawie.
Czy już,Czy już nie ma.,Czy już w tej chwili jest to że nie ma.,Czy już w tej sprawie.,Czy już jest w stanie nietrzeźwości.,Czy już po raz kolejny w tej sprawie.,Czy już w tej chwili jest to że nie ma.,Czy już dzisiaj nie ma.
Nie można,Nie można było w tym zakresie.,Nie można było w przypadku gdy w innych krajach.,Nie można było w tym zakresie.,Nie można było to że w tym zakresie.,Nie można było w tym zakresie.,Nie można było w tym zakresie.,Nie można było w polsce.
,W związku z tym że nie ma.,W związku z tym że nie ma.,W związku z tym że nie ma.,W związku z tym że nie ma.,W związku z tym że nie ma.,W związku z tym że nie ma.,W imieniu klubu parlamentarnego samoobrona rzeczypospolitej polskiej mam zaszczyt przedstawić stanowisko klubu w sprawie ustawy o zmianie.


In [20]:
pd.options.display.max_colwidth=500


In [34]:
Counter(model_PiS.model_dict['ja nie']).most_common(1)

[('wiem', 54)]

In [27]:
Counter(model_PO.model_dict['ja']).most_common(1)

[('nie', 439)]

In [28]:
Counter(model_SO.model_dict['ja']).most_common(1)

[('nie', 152)]

In [44]:
PiS = ' '.join(sp_joined['speech_rawer'][sp_joined.party_final=='Prawo i Sprawiedliwość'])
model_PiS = ng_models.ng_models(2, PiS)

In [45]:
model_PiS.mprob_sent('ja nie')

1  <s> <s> ja nie
2  ja nie
3  wiem


'Ja nie wiem czy pan minister powiedział że w tym zakresie.'